In [1]:
#Install all dependencies
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install jiwer
!pip install accelerate>=0.20.1 -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.5 MB/s eta 0:00:00


In [2]:
#import required libraries
from datasets import load_dataset, load_metric, Audio, concatenate_datasets
import re
import json
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import numpy as np

In [3]:
#login to huggingface
from huggingface_hub import notebook_login
notebook_login()

In [6]:
!pip install lightgbm==3.3.0
!pip install -i https://test.pypi.org/simple/ lang2vec
%cd /
!git clone https://github.com/neulab/langrank.git
%cd langrank
!pip install -r requirements.txt
!wget http://phontron.com/data/langrank/indexed.tar.gz  .
!tar -xzvf indexed.tar.gz
!rm indexed.tar.gz

Looking in indexes: https://test.pypi.org/simple/
/
Cloning into 'langrank'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 204 (delta 1), reused 4 (delta 1), pack-reused 198
Receiving objects: 100% (204/204), 3.55 MiB | 7.54 MiB/s, done.
Resolving deltas: 100% (100/100), done.
/langrank
--2023-11-02 01:04:17--  http://phontron.com/data/langrank/indexed.tar.gz
Resolving phontron.com (phontron.com)... 208.113.196.149
Connecting to phontron.com (phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129000607 (123M) [application/gzip]
Saving to: ‘indexed.tar.gz’

indexed.tar.gz      100%[===================>] 123.02M  44.9MB/s    in 2.7s    

2023-11-02 01:04:20 (44.9 MB/s) - ‘indexed.tar.gz’ saved [129000607/129000607]

--2023-11-02 01:04:20--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve 

In [7]:
import langrank as lr
lines = load_dataset("mozilla-foundation/common_voice_13_0", "gn", split="train", use_auth_token=True)['sentence']
prepared = lr.prepare_new_dataset('gug', task="MT", dataset_source=lines)
lr.rank(prepared, task='MT', candidates='all')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1414it [00:00, 56752.46it/s]


...


Reading metadata...: 5426it [00:00, 77142.84it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 142it [00:00, 112886.88it/s]


NOTE: no subword-level dataset provided, will only extract word-level features.
Preparing candidate list...
...geographic
...genetic
...inventory
...syntactic
...phonological
...featural
Loading model...
predicting...
Ranking with single features:
Overlap word-level
0. ted_glg : score=0.00
1. ted_vie : score=0.00
2. ted_bos : score=0.00
Overlap subword-level
0. ted_msa : score=0.00
1. ted_ita : score=0.00
2. ted_vie : score=0.00
Transfer lang dataset size
0. ted_ara : score=214111.00
1. ted_heb : score=211820.00
2. ted_rus : score=208458.00
Transfer over target size ratio
0. ted_ara : score=151.42
1. ted_heb : score=149.80
2. ted_rus : score=147.42
Transfer target TTR distance
0. ted_mya : score=0.00
1. ted_kaz : score=0.08
2. ted_tam : score=0.13
GENETIC
0. ted_eus : score=1.00
1. ted_bel : score=1.00
2. ted_tur : score=1.00
SYNTACTIC
0. ted_slk : score=0.66
1. ted_mon : score=0.66
2. ted_msa : score=0.66
FEATURAL
0. ted_spa : score=0.50
1. ted_eus : score=0.60
2. ted_tam : score=0.60

[19,
 17,
 30,
 10,
 33,
 21,
 32,
 3,
 16,
 49,
 11,
 45,
 23,
 27,
 5,
 51,
 22,
 1,
 38,
 43,
 47,
 50,
 41,
 37,
 9,
 34,
 44,
 4,
 13,
 36,
 6,
 53,
 8,
 15,
 20,
 39,
 28,
 52,
 35,
 48,
 12,
 42,
 40,
 7,
 2,
 18,
 26,
 14,
 0,
 46,
 31,
 24,
 25,
 29]

# Data Loading and Preprocessing

In [8]:
#Load the datasets using huggingface datasets
#The splits can be changed to train+validation, test to get the test WER
train_dataset_gn = load_dataset("mozilla-foundation/common_voice_13_0", "gn", split="train", use_auth_token=True)
dev_dataset = load_dataset("mozilla-foundation/common_voice_13_0", "gn", split="validation", use_auth_token=True)
train_dataset_hun = load_dataset("mozilla-foundation/common_voice_13_0", "hu", split=f"train[:{len(train_dataset_gn)}]", use_auth_token=True)
# dev_dataset_hun = load_dataset("mozilla-foundation/common_voice_13_0", "hu", split="validation", use_auth_token=True)

train_dataset = concatenate_datasets([train_dataset_gn, train_dataset_hun])
# dev_dataset = concatenate_datasets([dev_dataset_gn, dev_dataset_hun])

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 15516it [00:00, 90727.23it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...


Reading metadata...: 7875it [00:00, 107997.76it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 23308it [00:00, 123667.43it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1838it [00:00, 73768.06it/s]


In [9]:
#Filter the required columns
train_dataset = train_dataset.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes", "variant"])
dev_dataset = dev_dataset.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes", "variant"])

### Text preprocessing

In [10]:
#preprocess the text in the dataset
def preprocess_text(batch):
    batch["sentence"] = re.sub('[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']', '', batch["sentence"])
    batch["sentence"] = batch["sentence"].lower()
    return batch

train_dataset = train_dataset.map(preprocess_text)
dev_dataset = dev_dataset.map(preprocess_text)

Map:   0%|          | 0/2828 [00:00<?, ? examples/s]

Map:   0%|          | 0/352 [00:00<?, ? examples/s]

In [11]:
#construct the vocabulary and store it as a json file
def construct_vocab(batch):
  text = " ".join(batch["sentence"])
  vocab = list(set(text))
  return {"vocab": [vocab], "text": [text]}

vocab_train = train_dataset.map(construct_vocab, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_dev = dev_dataset.map(construct_vocab, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dev_dataset.column_names)

word_list = list(set(vocab_train["vocab"][0]) | set(vocab_dev["vocab"][0]))
vocab = {v: k for k, v in enumerate(sorted(word_list))}
vocab["[UNK]"] = len(vocab)
vocab["[PAD]"] = len(vocab)

with open('vocab.json', 'w') as f:
    json.dump(vocab, f)


Map:   0%|          | 0/2828 [00:00<?, ? examples/s]

Map:   0%|          | 0/352 [00:00<?, ? examples/s]

### Tokenizer and processor

In [12]:
#Tokenizer
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token=" ")

#Feature Extractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

#Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### Audio preprocessing

In [13]:
#Sample the audio in the dataset for preprocessing
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16_000))
dev_dataset = dev_dataset.cast_column("audio", Audio(sampling_rate=16_000))

def prepare_audio(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

train_dataset = train_dataset.map(prepare_audio, remove_columns=train_dataset.column_names)
dev_dataset = dev_dataset.map(prepare_audio, remove_columns=dev_dataset.column_names)

Map:   0%|          | 0/2828 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/352 [00:00<?, ? examples/s]

# Model Traning

### Data Collator

In [14]:
#Create a data collator class to pad the inputs to a fixed length
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

### Evaluation metric

In [15]:
def compute_wer(pred):
    wer_metric = load_metric("wer")
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Model parameters and training arguments

In [16]:
#Define the pretrained model
#Change the hyper-parameters here if required
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.05,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

#Training the entire model can be costly. Hence freeze the first few layers of the model that extract meaningful features from the data
model.freeze_feature_extractor()

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'lm_head.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1925: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [17]:
#Define the training arguments
#Change the hyper-parameters here if required
training_args = TrainingArguments(
  output_dir="./",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=False,
)

In [18]:
#Create a trainer instance that defined the model, data collator, datasets, tokenizer, training arguments and evaluation metrics
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_wer,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=processor.feature_extractor,
)

In [19]:
#Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
400,5.627400,3.634479,1.000000
800,1.054600,1.225514,0.894986
1200,0.239300,1.695879,0.943089
1600,0.143900,1.311910,0.829268
2000,0.100600,1.374923,0.820461
2400,0.072900,1.917837,0.907182


<ipython-input-15-dd986202b44b>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Tran

TrainOutput(global_step=2640, training_loss=1.1021072683912334, metrics={'train_runtime': 8518.2212, 'train_samples_per_second': 9.96, 'train_steps_per_second': 0.31, 'total_flos': 1.0909737114586606e+19, 'train_loss': 1.1021072683912334, 'epoch': 29.83})

# Model Evaluation

In [20]:
#Load and preprocess the test dataset
test_dataset = load_dataset("mozilla-foundation/common_voice_13_0", "gn", split="test", use_auth_token=True)
test_dataset = test_dataset.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes", "variant"])
test_dataset = test_dataset.map(preprocess_text)
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16_000))
test_dataset = test_dataset.map(prepare_audio, remove_columns=test_dataset.column_names)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(


Map:   0%|          | 0/811 [00:00<?, ? examples/s]

Map:   0%|          | 0/811 [00:00<?, ? examples/s]

In [21]:
#Pass it through the finetuned model an compute WER
#Please note the Following cells are just being executed on one row of the test data
from tqdm import tqdm
test_dataset_ref = load_dataset("mozilla-foundation/common_voice_13_0", "gn", split="test", token=True)
wer_metric = load_metric("wer")
wers = []
for input_value, sentence in tqdm(zip(test_dataset["input_values"], test_dataset_ref["sentence"])):
  input_dict = processor(input_value, return_tensors="pt", padding=True)
  logits = model(input_dict.input_values.to("cuda")).logits


  pred_ids = torch.argmax(logits.cpu(), dim=-1)

  label_str = sentence.lower()
  pred_str = processor.batch_decode(pred_ids)

  #print(pred_str, label_str)
  wers.append(wer_metric.compute(predictions=[pred_str], references=[label_str]))

print("wer", np.mean(wers))

0it [00:00, ?it/s]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
2it [00:00,  6.08it/s]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to p

wer 0.9248331473978822
